# Titanic تحلیل و مصورسازی دیتاست 
**خلاصه:** در این نوت‌بوک داده‌ها پاک‌سازی، ویژگی‌سازی و مصورسازی شدند.  
**هدف:** استخراج insight کلیدی و آماده‌سازی برای مدل‌سازی.

In [ ]:
import pandas as pd
import glob
from matplotlib import rcParams

# انتخاب فونت فارسی نصب شده روی سیستم
rcParams['font.family'] = 'Tahoma'

files = glob.glob("data/*.csv")

df = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

display(df.head())
display(df.info())

display(df.isnull().sum())

### وضعیت اولیه داده
- تعداد ردیف‌ها و ستون‌ها: `df.shape`
- ستون‌ها با بیشترین missing: `Embarked`, `Cabin`, `Age`, ...
- هدف cleaning: حذف/تبدیل ستون‌های غیرضروری، پر کردن مقادیر گمشده، اصلاح نوع داده.

In [ ]:
# قبل: درصد missing
cab_pct = df['Cabin'].isnull().mean()
cab_pct

**گام:** حذف ستون `Cabin`  
**دلیل:** بیش از 70% مقادیر گم‌شده دارد، و نگهداری آن کم‌فایده است (مگر اینکه از آن feature دیگری استخراج کنیم).  
**عمل:** `df.drop('Cabin', axis=1, inplace=True)`


In [ ]:
median_age = df['Age'].median()
df['Age'] = df['Age'].fillna(median_age)

**گام:** پر کردن `Age`  
**دلیل:** توزیع سن به شدت صفر نیست و outlier دارد → استفاده از میانه بهتر از میانگین است.  
**عمل:** مقدار گمشده با میانه (`{median_age}`) پر شد.

In [ ]:
df['Sex_num'] = df['Sex'].map({'male':0, 'female':1})

**گام:** تبدیل `Sex` به عدد  
**دلیل:** برای محاسبهٔ همبستگی و مدل‌سازی باید ویژگی‌های دسته‌ای کدگذاری شوند.  
**عمل:** `male`→0, `female`→1


In [ ]:
# FamilySize و IsAlone
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# Title از Name
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
df['Title'] = df['Title'].replace(['Mlle','Ms'],'Miss').replace('Mme','Mrs')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(16,4))

# Countplot: Sex vs Survived
plt.subplot(1,3,1)
sns.countplot(data=df, x='Sex', hue='Survived')
plt.title('Sex vs Survived')

# Boxplot: Pclass vs Fare
plt.subplot(1,3,2)
sns.boxplot(data=df, x='Pclass', y='Fare')
plt.title('Pclass vs Fare')

# Heatmap: numeric correlation
plt.subplot(1,3,3)
numeric = df.select_dtypes(include='number')
sns.heatmap(numeric.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation (numeric)')

plt.tight_layout()
plt.show()


### Insights (نمودارهای نهایی)
- **جنسیت:** زنان نسبت به مردان شانس نجات بیشتری دارند (مشاهده از Countplot).  
- **کلاس و کرایه:** کلاس بالاتر معمولاً کرایه بیشتر و شانس نجات بالاتری دارد (Boxplot + correlation).  
- **همبستگی:** `Survived` با `Fare` رابطه‌ی مثبت و با `Pclass` رابطه‌ی منفی دارد (Heatmap).

In [ ]:
log = [
    {'step':'drop Cabin', 'action':'drop', 'reason':'>70% missing'},
    {'step':'fill Age', 'action':'fillna', 'reason':'median used'},
    {'step':'encode Sex', 'action':'map', 'reason':'for modeling'},
    # ... ادامه
]
log_df = pd.DataFrame(log)
display(log_df)

### Cleaning Log
جدول بالا شامل لیست تغییرات، نوع عمل و دلیل آن است.

# جمع‌بندی نهایی (Insights)
1. **جنسیت:** زنان شانس نجات بیشتری داشتند — باید `Sex` را در مدل بگنجانیم.  
2. **کلاس مسافر (Pclass):** کلاس 1 شانس نجات بیشتر؛ Pclass و Fare مهم‌اند.  
3. **خانواده:** کسانی که تنها سفر می‌کردند (`IsAlone==1`) احتمال نجاتشان کمتر بود.  
4. **سن (Age):** سن تأثیر مستقیم کمتری نسبت به موارد بالا دارد، ولی کودکان ممکن است رفتار متفاوتی داشته باشند — (در صورت نیاز دسته‌بندی سن مناسب).  
5. **محدودیت‌ها:** ستون `Cabin` حذف شد به‌خاطر missing زیاد؛ ممکن است با استخراج ویژگی‌های بهتر از Cabin (مثل حرف اول) اطلاعات مفید بدست آید.


In [ ]:
from IPython.display import Markdown, display
summary_md = """
# Titanic Analysis Summary

## Cleaning Steps
- drop Cabin (reason: >70% missing)
- fill Age with median
...

## Key Insights
- ...
"""
display(Markdown(summary_md))

# ذخیره به فایل md
with open('titanic_summary.md','w',encoding='utf-8') as f:
    f.write(summary_md)
